# Web scraping tool (NB)

In this document lot of nice thing will happen :)

In [2]:
from urllib.request import urlopen
import requests
import re
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
import openai
import MACHINE_CONFIG
import newspaper
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from collections import defaultdict

In [6]:
import scraper
import os
CONFIG_PATH = r"C:\Users\andra\OneDrive\Dokumentumok\Munka\WeSpeakAi\GoodPeople\site-config.json"

In [11]:
scraper.site_selector(CONFIG_PATH)

## 0. Search for the page articles

Our first task is to collect every article from the site, but only the articles that we want!

In [30]:
def get_articles(base_url, article_rule, max_url = 1000, output_path=None, verbal=False):
    # Base URL of the website to crawl
    visited_urls = set()
    articles = set()
    urls_to_visit = deque([base_url])

    # Function to check if a URL belongs to the base domain
    def is_same_domain(url, base_domain):
        return urlparse(url).netloc == base_domain

    # Starting the crawl
    while urls_to_visit and len(articles)<max_url:
        current_url = urls_to_visit.popleft()
        
        # Skip if already visited
        if current_url in visited_urls:
            continue
        
        # Mark as visited
        visited_urls.add(current_url)
        if verbal: print(f"Visiting: {current_url}")
        
        try:
            response = requests.get(current_url)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                links = soup.find_all('a')
                filtered_links = [s.get('href') for s in links if (not s.get('href') is None and (s.get('href').startswith('/') or s.get('href').startswith(base_url)))]

                for link in filtered_links:
                    if link:
                        full_url = urljoin(base_url, link)
                        if is_same_domain(full_url, urlparse(base_url).netloc):
                            urls_to_visit.append(full_url)
                        if full_url.startswith(str(base_url+article_rule)): 
                            articles.add(full_url)
                            print(f"Count: [{len(articles)} of {max_url}]", flush=True)
            else:
                print(f"Failed to fetch {current_url}. Status code: {response.status_code}")
        except requests.RequestException as e:
            print(f"Error fetching {current_url}: {e}")
    if output_path:
        with open(output_path, 'w') as file:
            # Iterate through the list of URLs
            for url in articles:
                # Write each URL to the file followed by a newline character
                file.write(url + '\n')
    
    return articles

def has_common_element(list1, list2):
    # Convert both lists to sets
    set1 = set(list1)
    set2 = set(list2)
    
    # Check if there is any intersection between the sets
    return not set1.isdisjoint(set2)

def extract_and_count_urls(urls, lenght_of_posible_article):
    # Dictionary to hold the occurrences of each segment
    segment_counts = defaultdict(int)
    
    # List to store lists of segments for each URL
    all_segments = []
    articles = []
    
    for url in urls:
        # Parse the URL and extract the path
        parsed_url = urlparse(url)
        path = parsed_url.path
        segments = []
        # Split the path into segments and filter out empty segments
        for segment in path.split('/'):
            if segment:
                if len(segment) <= lenght_of_posible_article:
                    segments.append(segment)
                else:
                    segments.append("possible_article")
                    articles.append(parsed_url)
        # segments = [segment if len(segment) <= lenght_of_posible_article else "possible_article" for segment in path.split('/') if segment]
        
        # Append the list of segments for this URL to all_segments
        all_segments.append(segments)
        
        # Count each segment
        for segment in segments:
            segment_counts[segment] += 1
    
    # Convert the dictionary to a list of lists format
    segment_occurrences = [[segment, count] for segment, count in segment_counts.items()]
    
    return all_segments, segment_occurrences, articles

def visualize_segment_occurrences(segment_occurrences):
    # Sort the segments by their count in descending order
    segment_occurrences.sort(key=lambda x: x[1], reverse=True)
    
    # Unzip the list of lists into two lists
    segments, counts = zip(*segment_occurrences)
    
    # Create a bar plot
    plt.figure(figsize=(12, 6))
    plt.bar(segments, counts, color='skyblue')
    
    # Add title and labels
    plt.title('Occurrences of URL Segments')
    plt.xlabel('Segments')
    plt.ylabel('Occurrences')
    
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right')
    
    # Show the plot
    plt.tight_layout()
    plt.show()

def short_urls(urls, categories, cfp=False,  min_article_lenght=12):
    """
    cfp: Chategory is first in path
    """
    articles = []
    for url in urls:
        # Parse the URL and extract the path
        parsed_url = urlparse(url)
        path = parsed_url.path
        segments = [segment for segment in path.split('/') if segment]
        has_chategory = (segments[0] in categories) if cfp and len(segments)>0 else has_common_element(categories, segments)
        filtered_segments = [element for element in segments if element not in categories]
        has_posible_article_lenght = any([len(element) >= min_article_lenght for element in filtered_segments if len(element) >= min_article_lenght])
        if has_chategory and has_posible_article_lenght:
            articles.append(parsed_url)
    return articles
     
categories = ["environment", "society", "lifestyle", "science", "economics", "perspective", "opinion"]

#get_articles("https://reasonstobecheerful.world/", "", 100, r"test.txt")
# # Example usage:
with open(r"test.txt", "r") as file:
    urls = file.readlines()

# Extract segments and their occurrences
segments, occurrences, articles = extract_and_count_urls(urls, 10)

# articles = short_urls(urls, categories, True, 12)

# Print extracted segments and occurrences
print("Segments extracted from each URL:", segments)
print("Occurrences of each segment:", occurrences)
print(f"Articles: {len(articles)}")
for article in articles:
    print(article.netloc + article.path)

# Visualize the occurrences
visualize_segment_occurrences(occurrences)

In [ ]:
headers = {[
            {
              "name": ":authority",
              "value": "backend.thehappybroadcast.com"
            },
            {
              "name": ":method",
              "value": "POST"
            },
            {
              "name": ":path",
              "value": "/graphql"
            },
            {
              "name": ":scheme",
              "value": "https"
            },
            {
              "name": "accept",
              "value": "application/json"
            },
            {
              "name": "accept-encoding",
              "value": "gzip, deflate, br, zstd"
            },
            {
              "name": "accept-language",
              "value": "en-GB,en;q=0.9,en-US;q=0.8,hu;q=0.7,es;q=0.6"
            },
            {
              "name": "authorization",
              "value": "Bearer thb2021tokenW3b"
            },
            {
              "name": "content-length",
              "value": "377"
            },
            {
              "name": "content-type",
              "value": "application/json"
            },
            {
              "name": "dnt",
              "value": "1"
            },
            {
              "name": "origin",
              "value": "https://www.thehappybroadcast.com"
            },
            {
              "name": "priority",
              "value": "u=1, i"
            },
            {
              "name": "referer",
              "value": "https://www.thehappybroadcast.com/"
            },
            {
              "name": "sec-ch-ua",
              "value": "\"Not/A)Brand\";v=\"8\", \"Chromium\";v=\"126\", \"Microsoft Edge\";v=\"126\""
            },
            {
              "name": "sec-ch-ua-mobile",
              "value": "?0"
            },
            {
              "name": "sec-ch-ua-platform",
              "value": "\"Windows\""
            },
            {
              "name": "sec-fetch-dest",
              "value": "empty"
            },
            {
              "name": "sec-fetch-mode",
              "value": "cors"
            },
            {
              "name": "sec-fetch-site",
              "value": "same-site"
            },
            {
              "name": "user-agent",
              "value": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0"
            }
          ]}

url_search='https://backend.thehappybroadcast.com/graphql'
infos = {
    "queryString": [],
    "cookies": [],
    "headersSize": -1,
    "bodySize": 377,
    "postData": {
        "mimeType": "application/json",
        "text": "{\"query\":\"\\nquery getArchive($limit: Int, $start: Int) {\\n  archive {\\n    seo {\\n      title\\n      description\\n    }\\n  }\\n  posts(limit: $limit, start: $start, sort: \\\"date:desc\\\") {\\n    title\\n    slug\\n    date\\n    color\\n    caption\\n    image {\\n      url\\n      width\\n      height\\n    }\\n  } \\n}\\n\",\"variables\":{\"limit\":50,\"start\":100,\"dateGt\":null,\"dateLt\":null}}"
    }
}

data={
    "mimeType": "application/json",
    "text": "{\"query\":\"\\nquery getArchive($limit: Int, $start: Int) {\\n  archive {\\n    seo {\\n      title\\n      description\\n    }\\n  }\\n  posts(limit: $limit, start: $start, sort: \\\"date:desc\\\") {\\n    title\\n    slug\\n    date\\n    color\\n    caption\\n    image {\\n      url\\n      width\\n      height\\n    }\\n  } \\n}\\n\",\"variables\":{\"limit\":50,\"start\":100,\"dateGt\":null,\"dateLt\":null}}"
}



In [ ]:
s=requests.session()

req=s.post(
    url=url_search,
    headers=headers,
    json=data)

In [9]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
model = AutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")

In [10]:
path = r"C:\Users\andra\OneDrive\Dokumentumok\Munka\WeSpeakAi\GoodPeople\data\mbh\6845f0f8-32e9-11ef-9b29-8272bb82d234.md"
with open(path, "r") as file:
    text = file.read()

print(type(text))

**Bizonyos életkörülmények között egy nagyobb lakás vagy autó megvásárlása nem várathat magára, még akkor sem, ha ehhez viszonylag magas kamatozású hitelre is szükségünk van. A gazdaság ciklikussága miatt azonban viszonylag rövid távon előállhat olyan helyzet, melyben már a korábbinál sokkal kedvezőbb kamatok mellett kaphatunk kölcsönt, ezért célszerű a régi hitelt lecserélni egy új, kedvezőbb konstrukcióra. Mivel napjainkban ehhez hasonló folyamatok zajlanak a hitelpiacon, az MBH Bank szakértői összefoglalták, mikor érdemes kiváltanunk régi hiteleinket.**

Hitelfelvétel szempontjából most, 2024 májusában kedvező a helyzet, hiszen a gazdasági élet helyre állásával, a pénzügyi válság lecsengésével azt látjuk, hogy lassan egy éve csökkennek a hitelkamatok. Míg 2023. elején a lakáshitel kamatok 8-9% körül mozogtak, azok mára 6-7% körüli szintre mérséklődtek. De hasonló kamatesést figyelhettünk meg ezen távon  a fogyasztási hitelek esetében is. Igaz, a kamatszint még mindig nem tért vissza

In [ ]:
from keyphrasetransformer import KeyPhraseTransformer
kp = KeyPhraseTransformer()
kp.get_key_phrases(txt)

In [8]:
from keybert import KeyBERT

# Define a function to extract keywords
def extract_keywords(text, num_keywords=5):
  """
  Extracts keywords from a given text using KeyBERT.

  Args:
      text: The text to extract keywords from.
      num_keywords: The desired number of keywords to extract (default: 5).

  Returns:
      A list of the top keywords extracted from the text.
  """

  # Create a KeyBERT model
  ke = KeyBERT(model=)

  # Extract keywords
  keywords = ke.extract_keywords(text, top_n=num_keywords)

  return keywords

# Example usage
text = "This is a sample text to extract keywords. KeyBERT is a great tool for keyword extraction tasks."
keywords = extract_keywords(text)

print("Extracted Keywords:")
for keyword in keywords:
  print(keyword)


c:\Users\andra\miniconda3\envs\wsa\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\andra\miniconda3\envs\wsa\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\andra\miniconda3\envs\wsa\lib\runpy.py", line 86, in _run_code
    exec(code, run

RuntimeError: Numpy is not available

## 1. Scraping with JINA-AI-READER

*source:* [Jina](https://jina.ai/)

In [25]:
def jina_reader(url, save=False, output=None):
    """
        The response from JINA has the following structure:
        {
            satus: 200
            "data": {
                "title": The title of the article,
                "url": URL of the article
                "content": The article text
                "publishedTime": The publiseh time in '2024-06-17T10:34:05+00:00' format
            }        
        }

    """
    jina = 'https://r.jina.ai/'
    headers = {
    "Authorization": MACHINE_CONFIG.JINA_KEY,
    "Accept": "application/json"
    }

    response = requests.get(jina+url, headers=headers)
    response_text = response.text
    # Checking the status of the request
    if response.status_code == 200:
        response_body = response.content.decode('utf-8')
        response_body = json.loads(response_body)["data"]
        print(response_body)
        title = response_body["title"].strip().replace(" ", "_")
        text = response_body["content"]
    else:
        print(f'Failed to fetch data. Status code: {response.status_code}')
    if save:
        if  output is None: 
            output=str(title+".md")
        with open(output, "w", encoding="utf-8") as file:
            file.write(text)
    return response_body
    
def find_images_in_md_text(text):
    """
    # Function to find all image links in a Markdown file
    """
    image_pattern = r'!\[(.*?)\]\((.*?)\)'
    matches = re.findall(image_pattern, text)

    return matches

def find_images_on_page(url):
    article = newspaper.Article(url)
    article.download()
    article.parse()
    return article.images

def construct_article(url, save=False, output=None):
    data = jina_reader(url=url, save=save, output=output)
    images = find_images_in_md_text(data["content"])
    if len(images) == 0:
        images = find_images_on_page(url)
    #Extract title, timestamp
    article = {
        "url": url,
        "full_text": data["content"],
        "images": images,
        "title": data["title"],
        "time_stamp": data["publishedTime"]
    }
    if save:
        if  output is None: 
            output=str(data["title"].strip().replace(" ", "_")+".json")
        with open(output, "w", encoding="utf-8") as file:
            json.dump(article, file, ensure_ascii=False, indent=4)
    return article

DEMO

In [27]:
def test_jina_reader():
    url = "https://www.positive.news/lifestyle/life-after-escaping-gang-violence/"
    jina_reader(url, save=True, output="test.md")

test_jina_reader()

{'title': 'Life after: Escaping gang violence - Positive News', 'url': 'https://www.positive.news/lifestyle/life-after-escaping-gang-violence/', 'content': "Cenia Elizabeth Muñoz and her husband Angel feared becoming statistics of the widespread gang violence in their native El Salvador. After fleeing to the UK – arriving just before Covid-19 did – a university bridging course and a friendly welcome has helped the family to feel safe and hopeful once again\n\nWhen they lived in El Salvador, Cenia Elizabeth Muñoz and her husband Angel would turn off the lights every evening and hide from the gangs that operated near their house. “We had to be so quiet. If they know you are listening, or watching them, you are in trouble,” she says. “Our lives were always at risk.”\n\nOne night, the couple heard gunshots and crouched under their kitchen table to shelter while awaiting the police. “A young man had been killed near our house. We could hear his family crying. We could see his dead body lyin